#### Using PySpark to Train Tree Methods(Decision Trees, Random Forest, Gradient Boosted Trees) in Order to Predict Whether a College is Private or Public

By: Matt Purvis

In [0]:
# Import SparkSession and create session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tree').getOrCreate()

In [0]:
# Import the data
data = spark.sql('select * from College_csv')

In [0]:
# Preview the data
data.show()

+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
 School|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
Abilene Christian...| Yes|1660| 1232| 721| 23| 52| 2885| 537| 7440| 3300| 450| 2200| 70| 78| 18.1| 12| 7041| 60|
 Adelphi University| Yes|2186| 1924| 512| 16| 29| 2683| 1227| 12280| 6450| 750| 1500| 29| 30| 12.2| 16| 10527| 56|
 Adrian College| Yes|1428| 1097| 336| 22| 50| 1036| 99| 11250| 3750| 400| 1165| 53| 66| 12.9| 30| 8735| 54|
 Agnes Scott College| Yes| 417| 349| 137| 60| 89| 510| 63| 12960| 5450| 450| 875| 92| 97| 7.7| 37| 19016| 59|
Alaska Pacific Un...| Yes| 193| 146| 55| 16| 44| 249| 869| 7560| 4120| 800| 1500| 76| 72| 11.9| 2| 10922| 15|
 Albertson College| Yes| 587| 479| 158| 38| 62| 678| 41| 13500| 3335| 500| 675| 67| 73| 9.4| 11| 9727| 55|
Albertus Magnus C...| Yes| 353| 340| 103| 17| 45| 416| 230| 13290| 5720| 500| 1500| 90| 93| 11.5| 26| 8861| 63|
 Albion College| Yes|1899| 1720| 489| 37| 68| 1594| 32| 13868| 4826| 450| 850| 89| 100| 13.7| 37| 11487| 73|
 Albright College| Yes|1038| 839| 227| 30| 63| 973| 306| 15595| 4400| 300| 500| 79| 84| 11.3| 23| 11644| 80|
Alderson-Broaddus...| Yes| 582| 498| 172| 21| 44| 799| 78| 10468| 3380| 660| 1800| 40| 41| 11.5| 15| 8991| 52|
 Alfred University| Yes|1732| 1425| 472| 37| 75| 1830| 110| 16548| 5406| 500| 600| 82| 88| 11.3| 31| 10932| 73|
 Allegheny College| Yes|2652| 1900| 484| 44| 77| 1707| 44| 17080| 4440| 400| 600| 73| 91| 9.9| 41| 11711| 76|
Allentown Coll. o...| Yes|1179| 780| 290| 38| 64| 1130| 638| 9690| 4785| 600| 1000| 60| 84| 13.3| 21| 7940| 74|
 Alma College| Yes|1267| 1080| 385| 44| 73| 1306| 28| 12572| 4552| 400| 400| 79| 87| 15.3| 32| 9305| 68|
 Alverno College| Yes| 494| 313| 157| 23| 46| 1317| 1235| 8352| 3640| 650| 2449| 36| 69| 11.1| 26| 8127| 55|
American Internat...| Yes|1420| 1093| 220| 9| 22| 1018| 287| 8700| 4780| 450| 1400| 78| 84| 14.7| 19| 7355| 69|
 Amherst College| Yes|4302| 992| 418| 83| 96| 1593| 5| 19760| 5300| 660| 1598| 93| 98| 8.4| 63| 21424| 100|
 Anderson University| Yes|1216| 908| 423| 19| 40| 1819| 281| 10100| 3520| 550| 1100| 48| 61| 12.1| 14| 7994| 59|
 Andrews University| Yes|1130| 704| 322| 14| 23| 1586| 326| 9996| 3090| 900| 1320| 62| 66| 11.5| 18| 10908| 46|
Angelo State Univ...| No|3540| 2001| 1016| 24| 54| 4190| 1512| 5130| 3592| 500| 2000| 60| 62| 23.1| 5| 4010| 34|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
only showing top 20 rows

In [0]:
# Display column names and dtypes
data.printSchema()

root
-- School: string (nullable = true)
-- Private: string (nullable = true)
-- Apps: integer (nullable = true)
-- Accept: integer (nullable = true)
-- Enroll: integer (nullable = true)
-- Top10perc: integer (nullable = true)
-- Top25perc: integer (nullable = true)
-- F_Undergrad: integer (nullable = true)
-- P_Undergrad: integer (nullable = true)
-- Outstate: integer (nullable = true)
-- Room_Board: integer (nullable = true)
-- Books: integer (nullable = true)
-- Personal: integer (nullable = true)
-- PhD: integer (nullable = true)
-- Terminal: integer (nullable = true)
-- S_F_Ratio: double (nullable = true)
-- perc_alumni: integer (nullable = true)
-- Expend: integer (nullable = true)
-- Grad_Rate: integer (nullable = true)

In [0]:
# Import VectorAssembler in order to transform data into proper format
from pyspark.ml.feature import VectorAssembler

In [0]:
# View columns in order to choose features for assembler object
data.columns

Out[12]: ['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [0]:
# Create the assembler object that will be used to transform the data
assembler = VectorAssembler(inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'],outputCol='features')

In [0]:
# Transform the data and store in output variable
output = assembler.transform(data)

In [0]:
# Import StringIndexer to properly code Private column in ones and zeros
from pyspark.ml.feature import StringIndexer

In [0]:
# Create indexer object that will be used to code Private column
indexer = StringIndexer(inputCol='Private', outputCol = 'PrivateIndex')

In [0]:
# Code the Private column and store in output_fixed variable
output_fixed = indexer.fit(output).transform(output)

In [0]:
# Display the resulting df
output_fixed.show()

+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+--------------------+------------+
 School|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate| features|PrivateIndex|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+--------------------+------------+
Abilene Christian...| Yes|1660| 1232| 721| 23| 52| 2885| 537| 7440| 3300| 450| 2200| 70| 78| 18.1| 12| 7041| 60|[1660.0,1232.0,72...| 0.0|
 Adelphi University| Yes|2186| 1924| 512| 16| 29| 2683| 1227| 12280| 6450| 750| 1500| 29| 30| 12.2| 16| 10527| 56|[2186.0,1924.0,51...| 0.0|
 Adrian College| Yes|1428| 1097| 336| 22| 50| 1036| 99| 11250| 3750| 400| 1165| 53| 66| 12.9| 30| 8735| 54|[1428.0,1097.0,33...| 0.0|
 Agnes Scott College| Yes| 417| 349| 137| 60| 89| 510| 63| 12960| 5450| 450| 875| 92| 97| 7.7| 37| 19016| 59|[417.0,349.0,137....| 0.0|
Alaska Pacific Un...| Yes| 193| 146| 55| 16| 44| 249| 869| 7560| 4120| 800| 1500| 76| 72| 11.9| 2| 10922| 15|[193.0,146.0,55.0...| 0.0|
 Albertson College| Yes| 587| 479| 158| 38| 62| 678| 41| 13500| 3335| 500| 675| 67| 73| 9.4| 11| 9727| 55|[587.0,479.0,158....| 0.0|
Albertus Magnus C...| Yes| 353| 340| 103| 17| 45| 416| 230| 13290| 5720| 500| 1500| 90| 93| 11.5| 26| 8861| 63|[353.0,340.0,103....| 0.0|
 Albion College| Yes|1899| 1720| 489| 37| 68| 1594| 32| 13868| 4826| 450| 850| 89| 100| 13.7| 37| 11487| 73|[1899.0,1720.0,48...| 0.0|
 Albright College| Yes|1038| 839| 227| 30| 63| 973| 306| 15595| 4400| 300| 500| 79| 84| 11.3| 23| 11644| 80|[1038.0,839.0,227...| 0.0|
Alderson-Broaddus...| Yes| 582| 498| 172| 21| 44| 799| 78| 10468| 3380| 660| 1800| 40| 41| 11.5| 15| 8991| 52|[582.0,498.0,172....| 0.0|
 Alfred University| Yes|1732| 1425| 472| 37| 75| 1830| 110| 16548| 5406| 500| 600| 82| 88| 11.3| 31| 10932| 73|[1732.0,1425.0,47...| 0.0|
 Allegheny College| Yes|2652| 1900| 484| 44| 77| 1707| 44| 17080| 4440| 400| 600| 73| 91| 9.9| 41| 11711| 76|[2652.0,1900.0,48...| 0.0|
Allentown Coll. o...| Yes|1179| 780| 290| 38| 64| 1130| 638| 9690| 4785| 600| 1000| 60| 84| 13.3| 21| 7940| 74|[1179.0,780.0,290...| 0.0|
 Alma College| Yes|1267| 1080| 385| 44| 73| 1306| 28| 12572| 4552| 400| 400| 79| 87| 15.3| 32| 9305| 68|[1267.0,1080.0,38...| 0.0|
 Alverno College| Yes| 494| 313| 157| 23| 46| 1317| 1235| 8352| 3640| 650| 2449| 36| 69| 11.1| 26| 8127| 55|[494.0,313.0,157....| 0.0|
American Internat...| Yes|1420| 1093| 220| 9| 22| 1018| 287| 8700| 4780| 450| 1400| 78| 84| 14.7| 19| 7355| 69|[1420.0,1093.0,22...| 0.0|
 Amherst College| Yes|4302| 992| 418| 83| 96| 1593| 5| 19760| 5300| 660| 1598| 93| 98| 8.4| 63| 21424| 100|[4302.0,992.0,418...| 0.0|
 Anderson University| Yes|1216| 908| 423| 19| 40| 1819| 281| 10100| 3520| 550| 1100| 48| 61| 12.1| 14| 7994| 59|[1216.0,908.0,423...| 0.0|
 Andrews University| Yes|1130| 704| 322| 14| 23| 1586| 326| 9996| 3090| 900| 1320| 62| 66| 11.5| 18| 10908| 46|[1130.0,704.0,322...| 0.0|
Angelo State Univ...| No|3540| 2001| 1016| 24| 54| 4190| 1512| 5130| 3592| 500| 2000| 60| 62| 23.1| 5| 4010| 34|[3540.0,2001.0,10...| 1.0|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+--------------------+------------+
only showing top 20 rows

In [0]:
# Get only the features and the label column and store in new variable
final_data = output_fixed.select('features', 'PrivateIndex')

In [0]:
# Create train and test split
train_data, test_data = final_data.randomSplit([.7,.3])

In [0]:
# Import the classifiers we will use
from pyspark.ml.classification import DecisionTreeClassifier, GBTClassifier, RandomForestClassifier

In [0]:
# Create the decision tree, random forest and gradient boosted tree objects that will be used to train data
dtc = DecisionTreeClassifier(labelCol = 'PrivateIndex', featuresCol = 'features')
rfc = RandomForestClassifier(numTrees = 150, labelCol = 'PrivateIndex', featuresCol = 'features')
gbt = GBTClassifier(labelCol = 'PrivateIndex', featuresCol = 'features')

In [0]:
# Train all three models on the training data
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [0]:
# Get the predictions on the test data
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [0]:
dtc_preds.show()

+--------------------+------------+-------------+--------------------+----------+
 features|PrivateIndex|rawPrediction| probability|prediction|
+--------------------+------------+-------------+--------------------+----------+
[167.0,130.0,46.0...| 0.0| [258.0,0.0]| [1.0,0.0]| 0.0|
[232.0,182.0,99.0...| 0.0| [37.0,5.0]|[0.88095238095238...| 0.0|
[233.0,233.0,153....| 1.0| [37.0,5.0]|[0.88095238095238...| 0.0|
[244.0,198.0,82.0...| 0.0| [64.0,0.0]| [1.0,0.0]| 0.0|
[268.0,253.0,103....| 0.0| [258.0,0.0]| [1.0,0.0]| 0.0|
[280.0,143.0,79.0...| 0.0| [37.0,5.0]|[0.88095238095238...| 0.0|
[291.0,245.0,126....| 0.0| [64.0,0.0]| [1.0,0.0]| 0.0|
[323.0,278.0,122....| 0.0| [258.0,0.0]| [1.0,0.0]| 0.0|
[335.0,284.0,132....| 0.0| [258.0,0.0]| [1.0,0.0]| 0.0|
[342.0,254.0,126....| 0.0| [258.0,0.0]| [1.0,0.0]| 0.0|
[368.0,317.0,159....| 0.0| [258.0,0.0]| [1.0,0.0]| 0.0|
[368.0,344.0,212....| 1.0| [37.0,5.0]|[0.88095238095238...| 0.0|
[369.0,312.0,90.0...| 0.0| [258.0,0.0]| [1.0,0.0]| 0.0|
[372.0,362.0,181....| 0.0| [64.0,0.0]| [1.0,0.0]| 0.0|
[374.0,280.0,185....| 0.0| [258.0,0.0]| [1.0,0.0]| 0.0|
[377.0,358.0,181....| 0.0| [37.0,5.0]|[0.88095238095238...| 0.0|
[385.0,340.0,193....| 0.0| [64.0,0.0]| [1.0,0.0]| 0.0|
[404.0,400.0,169....| 0.0| [37.0,5.0]|[0.88095238095238...| 0.0|
[434.0,321.0,141....| 0.0| [258.0,0.0]| [1.0,0.0]| 0.0|
[434.0,412.0,319....| 1.0| [0.0,2.0]| [0.0,1.0]| 1.0|
+--------------------+------------+-------------+--------------------+----------+
only showing top 20 rows

In [0]:
rfc_preds.show()

+--------------------+------------+--------------------+--------------------+----------+
 features|PrivateIndex| rawPrediction| probability|prediction|
+--------------------+------------+--------------------+--------------------+----------+
[167.0,130.0,46.0...| 0.0|[148.728226895054...|[0.99152151263369...| 0.0|
[232.0,182.0,99.0...| 0.0|[121.586027061142...|[0.81057351374095...| 0.0|
[233.0,233.0,153....| 1.0|[104.769923883769...|[0.69846615922513...| 0.0|
[244.0,198.0,82.0...| 0.0|[138.128121131351...|[0.92085414087567...| 0.0|
[268.0,253.0,103....| 0.0|[148.655852355579...|[0.99103901570385...| 0.0|
[280.0,143.0,79.0...| 0.0|[116.948130643521...|[0.77965420429014...| 0.0|
[291.0,245.0,126....| 0.0|[137.604365797103...|[0.91736243864735...| 0.0|
[323.0,278.0,122....| 0.0|[148.790138958815...|[0.99193425972543...| 0.0|
[335.0,284.0,132....| 0.0|[148.652911575718...|[0.99101941050479...| 0.0|
[342.0,254.0,126....| 0.0|[148.774667223633...|[0.99183111482422...| 0.0|
[368.0,317.0,159....| 0.0|[145.564079437147...|[0.97042719624765...| 0.0|
[368.0,344.0,212....| 1.0|[99.5278974728073...|[0.66351931648538...| 0.0|
[369.0,312.0,90.0...| 0.0|[145.738626736250...|[0.97159084490833...| 0.0|
[372.0,362.0,181....| 0.0|[140.518416866740...|[0.93678944577826...| 0.0|
[374.0,280.0,185....| 0.0|[145.202184543289...|[0.96801456362193...| 0.0|
[377.0,358.0,181....| 0.0|[115.067910785175...|[0.76711940523450...| 0.0|
[385.0,340.0,193....| 0.0|[139.116779546469...|[0.92744519697646...| 0.0|
[404.0,400.0,169....| 0.0|[126.205046477225...|[0.84136697651483...| 0.0|
[434.0,321.0,141....| 0.0|[148.787283898209...|[0.99191522598806...| 0.0|
[434.0,412.0,319....| 1.0|[90.8445952962356...|[0.60563063530823...| 0.0|
+--------------------+------------+--------------------+--------------------+----------+
only showing top 20 rows

In [0]:
gbt_preds.show()

+--------------------+------------+--------------------+--------------------+----------+
 features|PrivateIndex| rawPrediction| probability|prediction|
+--------------------+------------+--------------------+--------------------+----------+
[167.0,130.0,46.0...| 0.0|[1.55127428658362...|[0.95699774887735...| 0.0|
[232.0,182.0,99.0...| 0.0|[0.53611095282553...|[0.74501923115389...| 0.0|
[233.0,233.0,153....| 1.0|[1.40176648816518...|[0.94286644116515...| 0.0|
[244.0,198.0,82.0...| 0.0|[1.53013695381043...|[0.95522401374060...| 0.0|
[268.0,253.0,103....| 0.0|[1.55127428658362...|[0.95699774887735...| 0.0|
[280.0,143.0,79.0...| 0.0|[1.45119182991817...|[0.94796414402476...| 0.0|
[291.0,245.0,126....| 0.0|[1.44823894722836...|[0.94767205178612...| 0.0|
[323.0,278.0,122....| 0.0|[1.50932555258579...|[0.95340964462186...| 0.0|
[335.0,284.0,132....| 0.0|[1.54542527071535...|[0.95651375721676...| 0.0|
[342.0,254.0,126....| 0.0|[1.54542527071535...|[0.95651375721676...| 0.0|
[368.0,317.0,159....| 0.0|[1.53678319203255...|[0.95578911973629...| 0.0|
[368.0,344.0,212....| 1.0|[0.95522717238510...|[0.87107018251861...| 0.0|
[369.0,312.0,90.0...| 0.0|[1.55442777845160...|[0.95725655373654...| 0.0|
[372.0,362.0,181....| 0.0|[1.49593193192707...|[0.95220520696491...| 0.0|
[374.0,280.0,185....| 0.0|[1.50065422496899...|[0.95263320322831...| 0.0|
[377.0,358.0,181....| 0.0|[1.18048311252112...|[0.91380194350741...| 0.0|
[385.0,340.0,193....| 0.0|[1.53601801739528...|[0.95572440774634...| 0.0|
[404.0,400.0,169....| 0.0|[1.13414294736754...|[0.90621621011538...| 0.0|
[434.0,321.0,141....| 0.0|[1.54542527071535...|[0.95651375721676...| 0.0|
[434.0,412.0,319....| 1.0|[-1.4138154323621...|[0.05584919119912...| 1.0|
+--------------------+------------+--------------------+--------------------+----------+
only showing top 20 rows

In [0]:
# Import evaluator to get ROC/AUC and create evaluator object
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_binary_eval = BinaryClassificationEvaluator(labelCol='PrivateIndex')

In [0]:
# Print ROC/AUC for Decision tree
print('DTC')
print(my_binary_eval.evaluate(dtc_preds))

DTC
0.8956995669702852

In [0]:
# Print ROC/AUC for Random Forest
print('RFC')
print(my_binary_eval.evaluate(rfc_preds))

RFC
0.9817828878602358

In [0]:
# Print ROC/AUC for Gradient Boosted Trees
print('GBT')
print(my_binary_eval.evaluate(gbt_preds))

GBT
0.9522174107809469

The Random Forest Classifier performed the best, followed by the Gradient Boosted Trees Classifier. The Decision Tree was the worse model, which makes sense since it only builds one tree. Further tweaking the model parameters for each of the models would probably boost the performance even more. Performing a grid search would be ideal in this scenario.

#### Looking at Accuracy

In [0]:
# import multiclass evaluator and create the object
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval = MulticlassClassificationEvaluator(labelCol = 'PrivateIndex', metricName = 'accuracy')

In [0]:
# Fit the evaluator on the predictions from each model
dtc_acc = acc_eval.evaluate(dtc_preds)
rfc_acc = acc_eval.evaluate(rfc_preds)
gbt_acc = acc_eval.evaluate(gbt_preds)

In [0]:
print('DTC')
dtc_acc

DTC
Out[73]: 0.9176470588235294

In [0]:
print('RFC')
rfc_acc

RFC
Out[74]: 0.9450980392156862

In [0]:
print('GBT')
gbt_acc

GBT
Out[75]: 0.9019607843137255

The random forest had the highest accuracy, followed by the decision tree and then the GBT classifier. It is important to not look at accuracy in a vaccuum. Unbalanced datasets can impact accuracy. We could also look at precision and recall and f1 score as well. All metrics should be evaluated together in order to get a full picture.